In [1]:
from lvq import *
from scipy.io import arff
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
import pandas as pd

In [2]:
data = arff.loadarff('datasets/jm1.arff')
df = pd.DataFrame(data[0])
df['defects'] = pd.factorize(df['defects'])[0] + 1
df_norm = (df - df.mean()) / (df.max() - df.min())
data = df_norm.values
np.random.shuffle(data)
prototypes = train_prototypes(data, 200, 0.3, 1)

0.3


In [ ]:
prototypes

In [ ]:
x_train = np.asarray(prototypes)[:,:-1]
y_train = np.asarray(prototypes)[:,-1]
x_test = np.asarray(data)[:,:-1]
y_test = np.asarray(data)[:,-1]

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1, n_jobs=4)
knn.fit(x_train, y_train)